# Data Download
Download data from http://maximedb.com to use in your research. The script will download all the documents available in the `corpus` directory.

🛑 Replace the `API_KEY` variable with your own key. If you don't have one you can get one for free on http://maximedb.com

In [27]:
import grequests
import requests
import os
import tempfile
import shutil
import zipfile
from utils import API_KEY

In [34]:
headers = {'Authorization': API_KEY}
base_0 = 'http://188.166.23.190/api/company_list'
base_1 = 'http://188.166.23.190/api/document_list'
base_2 = 'http://188.166.23.190/api/document'
base_3 = 'http://188.166.23.190/api/documents'
text_location = os.path.join(os.getcwd(), 'corpus')
print(text_location)

C:\Users\maxime\Documents\pytorch\maximedb\cnn2vec\corpus


Download the list of companies available in the database

In [29]:
r = requests.get(base_0, headers=headers)
company_list = r.json()

Make a list of the documents that you do not already have. 

In [39]:
document_to_dl = []
for i, company in enumerate(company_list):
    print('Downloading document list for {}'.format(company['name']))
    params = dict(query=company['company_id'], query_by='company_id')
    r = requests.get(base_1, params=params, headers=headers)
    if r.status_code != 200:
        print('Error for {}'.format(company['company_id']))
        continue
    document_list = r.json()
    filename = os.path.join(text_location, 'doc_{}.txt')
    for document in document_list:
        if not os.path.isfile(filename.format(document['document_id'])):
            document_to_dl.append(document['document_id'])       

KeyboardInterrupt
Thu Feb 15 21:25:19 2018


KeyboardInterrupt: 

Download the documents in series of zip files (each containing 1024 docs). Just like you would split a big file into multiple smaller files.

In [ ]:
chunk_size = 1024
for i in range(0, len(document_to_dl), chunk_size):
    print('Downloaded {:02}%)'.format(i/len(document_to_dl)))
    params = dict(ids=document_to_dl[i:i+chunk_size])
    r = requests.post(base_3, json=params, headers=headers, stream=True)
    hndl, tmp = tempfile.mkstemp(suffix=".zip")
    with open(tmp, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024*1000): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    zip_ref = zipfile.ZipFile(tmp, 'r')
    zip_ref.extractall(text_location)
    zip_ref.close()
    os.close(hndl)
    os.remove(tmp)